In [1]:
## This file implements neural networks and logistic regression on p0017presabs_quant.
## For fully-connected neural networks, the accuracy is 64.94%, and 66.23% after improvememt.
## To improve accuracy, we construct lasso regression to select important features, and thus form a new dataset.
## With the new dataset, the accuracy is 66.23%, and keeps 66.23% after improvement.
## For logistic regression, the accuracy is 62.34%.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 62.34%.
## For random forest with cross-validation, the mean accuracy is 75.26%.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017presabs_quant.csv')
df.shape

(255, 7004)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.871375
1      0.364375
2      0.803000
3      0.952625
4      1.012875
5      0.936750
6      0.745375
7      0.348375
8      0.571750
9      0.637125
10     0.670500
11     0.731250
12     0.865125
13     0.569375
14     0.327000
15     0.175750
16     0.964000
17     1.104167
18     0.223667
19     0.340167
20     0.438500
21     0.271833
22     0.248833
23     0.739333
24     0.232833
25     0.601500
26     0.251000
27     0.202833
28     0.599833
29     0.260667
         ...   
225    0.459500
226    0.825333
227    0.763167
228    0.334500
229    0.998833
230    0.491125
231    0.405750
232    0.875375
233    0.761500
234    0.255000
235    0.455125
236    0.234125
237    0.340125
238    0.952750
239    0.238500
240    0.294500
241    0.172125
242    0.221500
243    0.989875
244    0.763000
245    0.229875
246    0.215125
247    0.805375
248    0.184875
249    0.663250
250    0.803125
251    0.443875
252    0.207500
253    0.933875
254    0.873375
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.871375
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.364375
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.803000
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.952625
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1.012875


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

1    135
0    120
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 7004)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 7003)

In [13]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7002) (255,)


In [15]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [18]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [19]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 811us/step - loss: 0.6350 - accuracy: 0.6067 - val_loss: 0.6466 - val_accuracy: 0.5455
Epoch 2/100
178/178 [==============================] - 0s 188us/step - loss: 0.5690 - accuracy: 0.6798 - val_loss: 0.6667 - val_accuracy: 0.5455
Epoch 3/100
178/178 [==============================] - 0s 199us/step - loss: 0.5427 - accuracy: 0.7079 - val_loss: 0.6464 - val_accuracy: 0.5844
Epoch 4/100
178/178 [==============================] - 0s 207us/step - loss: 0.5203 - accuracy: 0.7584 - val_loss: 0.6593 - val_accuracy: 0.5974
Epoch 5/100
178/178 [==============================] - 0s 197us/step - loss: 0.5046 - accuracy: 0.7753 - val_loss: 0.6974 - val_accuracy: 0.6234
Epoch 6/100
178/178 [==============================] - 0s 194us/step - loss: 0.4997 - accuracy: 0.7528 - val_loss: 0.6619 - val_accuracy: 0.5974
Epoch 7/100
178/178 [==============================] - 0s 178us/step - loss: 0.4786 -

Epoch 57/100
178/178 [==============================] - 0s 162us/step - loss: 0.2959 - accuracy: 0.8820 - val_loss: 0.7655 - val_accuracy: 0.6364
Epoch 58/100
178/178 [==============================] - 0s 177us/step - loss: 0.2939 - accuracy: 0.8989 - val_loss: 0.7627 - val_accuracy: 0.6494
Epoch 59/100
178/178 [==============================] - 0s 172us/step - loss: 0.2955 - accuracy: 0.9045 - val_loss: 0.8018 - val_accuracy: 0.6364
Epoch 60/100
178/178 [==============================] - 0s 171us/step - loss: 0.3002 - accuracy: 0.8989 - val_loss: 0.7707 - val_accuracy: 0.6494
Epoch 61/100
178/178 [==============================] - 0s 170us/step - loss: 0.2910 - accuracy: 0.9045 - val_loss: 0.7779 - val_accuracy: 0.6364
Epoch 62/100
178/178 [==============================] - 0s 174us/step - loss: 0.2891 - accuracy: 0.8876 - val_loss: 0.7695 - val_accuracy: 0.6364
Epoch 63/100
178/178 [==============================] - 0s 171us/step - loss: 0.2826 - accuracy: 0.9157 - val_loss: 0.7655 -

In [21]:
acc_test = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))

77/77 [==============================] - 0s 111us/step
test accuracy: 64.94%


In [64]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [65]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [66]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 749us/step - loss: 0.6947 - accuracy: 0.4719 - val_loss: 0.6342 - val_accuracy: 0.6234
Epoch 2/1000
178/178 [==============================] - 0s 164us/step - loss: 0.6284 - accuracy: 0.6292 - val_loss: 0.6301 - val_accuracy: 0.5974
Epoch 3/1000
178/178 [==============================] - 0s 182us/step - loss: 0.6086 - accuracy: 0.6685 - val_loss: 0.6292 - val_accuracy: 0.6104
Epoch 4/1000
178/178 [==============================] - 0s 181us/step - loss: 0.5953 - accuracy: 0.7022 - val_loss: 0.6297 - val_accuracy: 0.6104
Epoch 5/1000
178/178 [==============================] - 0s 195us/step - loss: 0.5833 - accuracy: 0.7079 - val_loss: 0.6284 - val_accuracy: 0.6104
Epoch 6/1000
178/178 [==============================] - 0s 208us/step - loss: 0.5755 - accuracy: 0.7191 - val_loss: 0.6278 - val_accuracy: 0.6364
Epoch 7/1000
178/178 [==============================] - 0s 190us/step - loss: 0

178/178 [==============================] - 0s 214us/step - loss: 0.3830 - accuracy: 0.8483 - val_loss: 0.6973 - val_accuracy: 0.6234
Epoch 57/1000
178/178 [==============================] - 0s 184us/step - loss: 0.3798 - accuracy: 0.8483 - val_loss: 0.7022 - val_accuracy: 0.6234
Epoch 58/1000
178/178 [==============================] - 0s 192us/step - loss: 0.3810 - accuracy: 0.8483 - val_loss: 0.7000 - val_accuracy: 0.6234
Epoch 59/1000
178/178 [==============================] - 0s 171us/step - loss: 0.3762 - accuracy: 0.8483 - val_loss: 0.7011 - val_accuracy: 0.6234
Epoch 60/1000
178/178 [==============================] - 0s 182us/step - loss: 0.3731 - accuracy: 0.8539 - val_loss: 0.7074 - val_accuracy: 0.6234
Epoch 61/1000
178/178 [==============================] - 0s 205us/step - loss: 0.3708 - accuracy: 0.8539 - val_loss: 0.7035 - val_accuracy: 0.6234
Epoch 62/1000
178/178 [==============================] - 0s 164us/step - loss: 0.3701 - accuracy: 0.8539 - val_loss: 0.7130 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 180us/step - loss: 0.2838 - accuracy: 0.8989 - val_loss: 0.7873 - val_accuracy: 0.6494
Epoch 113/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2735 - accuracy: 0.9157 - val_loss: 0.7814 - val_accuracy: 0.6364
Epoch 114/1000
178/178 [==============================] - 0s 198us/step - loss: 0.2727 - accuracy: 0.8989 - val_loss: 0.7848 - val_accuracy: 0.6364
Epoch 115/1000
178/178 [==============================] - 0s 162us/step - loss: 0.2786 - accuracy: 0.8989 - val_loss: 0.8003 - val_accuracy: 0.6364
Epoch 116/1000
178/178 [==============================] - 0s 192us/step - loss: 0.2754 - accuracy: 0.9101 - val_loss: 0.7994 - val_accuracy: 0.6494
Epoch 117/1000
178/178 [==============================] - 0s 174us/step - loss: 0.2683 - accuracy: 0.9157 - val_loss: 0.8020 - val_accuracy: 0.6494
Epoch 118/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2685 - accuracy: 0.8989 - val_

178/178 [==============================] - 0s 169us/step - loss: 0.2107 - accuracy: 0.9213 - val_loss: 0.8952 - val_accuracy: 0.6364
Epoch 168/1000
178/178 [==============================] - 0s 178us/step - loss: 0.2066 - accuracy: 0.9270 - val_loss: 0.8962 - val_accuracy: 0.6364
Epoch 169/1000
178/178 [==============================] - 0s 174us/step - loss: 0.2183 - accuracy: 0.9101 - val_loss: 0.8892 - val_accuracy: 0.6364
Epoch 170/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2083 - accuracy: 0.9270 - val_loss: 0.9052 - val_accuracy: 0.6364
Epoch 171/1000
178/178 [==============================] - 0s 206us/step - loss: 0.2047 - accuracy: 0.9326 - val_loss: 0.9039 - val_accuracy: 0.6234
Epoch 172/1000
178/178 [==============================] - 0s 178us/step - loss: 0.2199 - accuracy: 0.9270 - val_loss: 0.8969 - val_accuracy: 0.6494
Epoch 173/1000
178/178 [==============================] - 0s 166us/step - loss: 0.2021 - accuracy: 0.9326 - val_loss: 0.9000 - 

178/178 [==============================] - 0s 190us/step - loss: 0.1771 - accuracy: 0.9326 - val_loss: 0.9809 - val_accuracy: 0.6494
Epoch 223/1000
178/178 [==============================] - 0s 181us/step - loss: 0.1621 - accuracy: 0.9494 - val_loss: 0.9803 - val_accuracy: 0.6364
Epoch 224/1000
178/178 [==============================] - 0s 179us/step - loss: 0.1737 - accuracy: 0.9438 - val_loss: 1.0013 - val_accuracy: 0.6364
Epoch 225/1000
178/178 [==============================] - 0s 170us/step - loss: 0.1583 - accuracy: 0.9551 - val_loss: 0.9898 - val_accuracy: 0.6364
Epoch 226/1000
178/178 [==============================] - 0s 159us/step - loss: 0.1615 - accuracy: 0.9551 - val_loss: 0.9879 - val_accuracy: 0.6234
Epoch 227/1000
178/178 [==============================] - 0s 173us/step - loss: 0.1704 - accuracy: 0.9438 - val_loss: 1.0278 - val_accuracy: 0.6364
Epoch 228/1000
178/178 [==============================] - 0s 186us/step - loss: 0.1846 - accuracy: 0.9326 - val_loss: 1.0043 - 

...

178/178 [==============================] - 0s 202us/step - loss: 0.0435 - accuracy: 0.9888 - val_loss: 1.7169 - val_accuracy: 0.6753
Epoch 773/1000
178/178 [==============================] - 0s 152us/step - loss: 0.0440 - accuracy: 0.9831 - val_loss: 1.6994 - val_accuracy: 0.6623
Epoch 774/1000
178/178 [==============================] - 0s 174us/step - loss: 0.0397 - accuracy: 0.9831 - val_loss: 1.6738 - val_accuracy: 0.6494
Epoch 775/1000
178/178 [==============================] - 0s 180us/step - loss: 0.0519 - accuracy: 0.9775 - val_loss: 1.6911 - val_accuracy: 0.6753
Epoch 776/1000
178/178 [==============================] - 0s 199us/step - loss: 0.0396 - accuracy: 0.9831 - val_loss: 1.6741 - val_accuracy: 0.6494
Epoch 777/1000
178/178 [==============================] - 0s 172us/step - loss: 0.0455 - accuracy: 0.9831 - val_loss: 1.6957 - val_accuracy: 0.6623
Epoch 778/1000
178/178 [==============================] - 0s 163us/step - loss: 0.0408 - accuracy: 0.9944 - val_loss: 1.7110 - 

178/178 [==============================] - 0s 168us/step - loss: 0.0353 - accuracy: 1.0000 - val_loss: 1.7531 - val_accuracy: 0.6623
Epoch 828/1000
178/178 [==============================] - 0s 153us/step - loss: 0.0363 - accuracy: 0.9831 - val_loss: 1.7232 - val_accuracy: 0.6494
Epoch 829/1000
178/178 [==============================] - 0s 182us/step - loss: 0.0412 - accuracy: 0.9944 - val_loss: 1.7527 - val_accuracy: 0.6623
Epoch 830/1000
178/178 [==============================] - 0s 180us/step - loss: 0.0391 - accuracy: 0.9775 - val_loss: 1.7667 - val_accuracy: 0.6753
Epoch 831/1000
178/178 [==============================] - 0s 182us/step - loss: 0.0358 - accuracy: 0.9944 - val_loss: 1.7485 - val_accuracy: 0.6623
Epoch 832/1000
178/178 [==============================] - 0s 163us/step - loss: 0.0364 - accuracy: 0.9888 - val_loss: 1.7732 - val_accuracy: 0.6623
Epoch 833/1000
178/178 [==============================] - 0s 185us/step - loss: 0.0379 - accuracy: 0.9831 - val_loss: 1.7589 - 

178/178 [==============================] - 0s 174us/step - loss: 0.0350 - accuracy: 0.9888 - val_loss: 1.7684 - val_accuracy: 0.6623
Epoch 883/1000
178/178 [==============================] - 0s 186us/step - loss: 0.0363 - accuracy: 0.9944 - val_loss: 1.7734 - val_accuracy: 0.6364
Epoch 884/1000
178/178 [==============================] - 0s 177us/step - loss: 0.0321 - accuracy: 0.9888 - val_loss: 1.8065 - val_accuracy: 0.6753
Epoch 885/1000
178/178 [==============================] - 0s 166us/step - loss: 0.0356 - accuracy: 0.9944 - val_loss: 1.7608 - val_accuracy: 0.6364
Epoch 886/1000
178/178 [==============================] - 0s 148us/step - loss: 0.0332 - accuracy: 0.9944 - val_loss: 1.8311 - val_accuracy: 0.6623
Epoch 887/1000
178/178 [==============================] - 0s 203us/step - loss: 0.0413 - accuracy: 0.9831 - val_loss: 1.8034 - val_accuracy: 0.6623
Epoch 888/1000
178/178 [==============================] - 0s 148us/step - loss: 0.0367 - accuracy: 0.9831 - val_loss: 1.7709 - 

178/178 [==============================] - 0s 176us/step - loss: 0.0362 - accuracy: 0.9831 - val_loss: 1.8481 - val_accuracy: 0.6753
Epoch 938/1000
178/178 [==============================] - 0s 187us/step - loss: 0.0318 - accuracy: 0.9888 - val_loss: 1.8055 - val_accuracy: 0.6364
Epoch 939/1000
178/178 [==============================] - 0s 168us/step - loss: 0.0299 - accuracy: 1.0000 - val_loss: 1.8427 - val_accuracy: 0.6623
Epoch 940/1000
178/178 [==============================] - 0s 163us/step - loss: 0.0302 - accuracy: 0.9888 - val_loss: 1.8326 - val_accuracy: 0.6623
Epoch 941/1000
178/178 [==============================] - 0s 197us/step - loss: 0.0324 - accuracy: 0.9944 - val_loss: 1.8283 - val_accuracy: 0.6623
Epoch 942/1000
178/178 [==============================] - 0s 178us/step - loss: 0.0293 - accuracy: 1.0000 - val_loss: 1.8368 - val_accuracy: 0.6623
Epoch 943/1000
178/178 [==============================] - 0s 147us/step - loss: 0.0293 - accuracy: 0.9944 - val_loss: 1.8410 - 

178/178 [==============================] - 0s 273us/step - loss: 0.0328 - accuracy: 0.9944 - val_loss: 1.8749 - val_accuracy: 0.6623
Epoch 993/1000
178/178 [==============================] - 0s 173us/step - loss: 0.0276 - accuracy: 0.9944 - val_loss: 1.8726 - val_accuracy: 0.6623
Epoch 994/1000
178/178 [==============================] - 0s 163us/step - loss: 0.0333 - accuracy: 0.9831 - val_loss: 1.8700 - val_accuracy: 0.6623
Epoch 995/1000
178/178 [==============================] - 0s 162us/step - loss: 0.0305 - accuracy: 0.9888 - val_loss: 1.8460 - val_accuracy: 0.6364
Epoch 996/1000
178/178 [==============================] - 0s 157us/step - loss: 0.0268 - accuracy: 1.0000 - val_loss: 1.8834 - val_accuracy: 0.6753
Epoch 997/1000
178/178 [==============================] - 0s 185us/step - loss: 0.0317 - accuracy: 0.9944 - val_loss: 1.8942 - val_accuracy: 0.6753
Epoch 998/1000
178/178 [==============================] - 0s 171us/step - loss: 0.0281 - accuracy: 0.9888 - val_loss: 1.8482 - 

In [67]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 259us/step
test accuracy: 66.23%


In [72]:
#### add regularizer and dropout
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [73]:
model3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [74]:
model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 1ms/step - loss: 2.2379 - accuracy: 0.5506 - val_loss: 0.8462 - val_accuracy: 0.5584
Epoch 2/1000
178/178 [==============================] - 0s 192us/step - loss: 2.0706 - accuracy: 0.5169 - val_loss: 0.7707 - val_accuracy: 0.5844
Epoch 3/1000
178/178 [==============================] - 0s 204us/step - loss: 2.2832 - accuracy: 0.4831 - val_loss: 0.7429 - val_accuracy: 0.5844
Epoch 4/1000
178/178 [==============================] - 0s 175us/step - loss: 2.2249 - accuracy: 0.5281 - val_loss: 0.7283 - val_accuracy: 0.5584
Epoch 5/1000
178/178 [==============================] - 0s 181us/step - loss: 2.2909 - accuracy: 0.5281 - val_loss: 0.7217 - val_accuracy: 0.5844
Epoch 6/1000
178/178 [==============================] - 0s 207us/step - loss: 2.6518 - accuracy: 0.4775 - val_loss: 0.7194 - val_accuracy: 0.5584
Epoch 7/1000
178/178 [==============================] - 0s 190us/step - loss: 1.6

178/178 [==============================] - 0s 232us/step - loss: 2.3512 - accuracy: 0.6067 - val_loss: 0.7000 - val_accuracy: 0.6234
Epoch 57/1000
178/178 [==============================] - 0s 236us/step - loss: 2.6732 - accuracy: 0.6011 - val_loss: 0.7008 - val_accuracy: 0.6234
Epoch 58/1000
178/178 [==============================] - 0s 199us/step - loss: 2.7034 - accuracy: 0.5674 - val_loss: 0.7010 - val_accuracy: 0.5974
Epoch 59/1000
178/178 [==============================] - 0s 202us/step - loss: 1.8286 - accuracy: 0.6517 - val_loss: 0.7011 - val_accuracy: 0.6104
Epoch 60/1000
178/178 [==============================] - 0s 177us/step - loss: 2.5796 - accuracy: 0.6180 - val_loss: 0.7007 - val_accuracy: 0.5974
Epoch 61/1000
178/178 [==============================] - 0s 177us/step - loss: 2.2587 - accuracy: 0.5955 - val_loss: 0.7021 - val_accuracy: 0.5844
Epoch 62/1000
178/178 [==============================] - 0s 165us/step - loss: 2.0316 - accuracy: 0.6124 - val_loss: 0.7014 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 194us/step - loss: 1.6891 - accuracy: 0.7022 - val_loss: 0.6975 - val_accuracy: 0.6623
Epoch 113/1000
178/178 [==============================] - 0s 167us/step - loss: 2.3093 - accuracy: 0.6404 - val_loss: 0.6960 - val_accuracy: 0.6623
Epoch 114/1000
178/178 [==============================] - 0s 181us/step - loss: 1.5493 - accuracy: 0.7191 - val_loss: 0.6967 - val_accuracy: 0.6623
Epoch 115/1000
178/178 [==============================] - 0s 179us/step - loss: 2.0318 - accuracy: 0.6798 - val_loss: 0.6943 - val_accuracy: 0.6623
Epoch 116/1000
178/178 [==============================] - 0s 167us/step - loss: 2.4439 - accuracy: 0.6742 - val_loss: 0.6948 - val_accuracy: 0.6623
Epoch 117/1000
178/178 [==============================] - 0s 181us/step - loss: 2.1443 - accuracy: 0.6517 - val_loss: 0.6976 - val_accuracy: 0.6623
Epoch 118/1000
178/178 [==============================] - 0s 182us/step - loss: 2.0637 - accuracy: 0.6685 - val_

178/178 [==============================] - 0s 200us/step - loss: 1.8340 - accuracy: 0.7921 - val_loss: 0.6914 - val_accuracy: 0.6364
Epoch 168/1000
178/178 [==============================] - 0s 175us/step - loss: 2.1479 - accuracy: 0.7753 - val_loss: 0.6903 - val_accuracy: 0.6364
Epoch 169/1000
178/178 [==============================] - 0s 167us/step - loss: 1.8076 - accuracy: 0.8034 - val_loss: 0.6907 - val_accuracy: 0.6364
Epoch 170/1000
178/178 [==============================] - 0s 168us/step - loss: 2.7689 - accuracy: 0.7191 - val_loss: 0.6905 - val_accuracy: 0.6364
Epoch 171/1000
178/178 [==============================] - 0s 197us/step - loss: 2.0754 - accuracy: 0.7978 - val_loss: 0.6924 - val_accuracy: 0.6364
Epoch 172/1000
178/178 [==============================] - 0s 168us/step - loss: 2.1472 - accuracy: 0.7809 - val_loss: 0.6928 - val_accuracy: 0.6364
Epoch 173/1000
178/178 [==============================] - 0s 177us/step - loss: 1.8180 - accuracy: 0.8034 - val_loss: 0.6913 - 

178/178 [==============================] - 0s 186us/step - loss: 1.9406 - accuracy: 0.8146 - val_loss: 0.6915 - val_accuracy: 0.6364
Epoch 223/1000
178/178 [==============================] - 0s 193us/step - loss: 2.2008 - accuracy: 0.7978 - val_loss: 0.6917 - val_accuracy: 0.6364
Epoch 224/1000
178/178 [==============================] - 0s 171us/step - loss: 1.7787 - accuracy: 0.8202 - val_loss: 0.6921 - val_accuracy: 0.6364
Epoch 225/1000
178/178 [==============================] - 0s 169us/step - loss: 2.3534 - accuracy: 0.7809 - val_loss: 0.6925 - val_accuracy: 0.6234
Epoch 226/1000
178/178 [==============================] - 0s 166us/step - loss: 2.4634 - accuracy: 0.7865 - val_loss: 0.6918 - val_accuracy: 0.6364
Epoch 227/1000
178/178 [==============================] - 0s 178us/step - loss: 2.2647 - accuracy: 0.7865 - val_loss: 0.6917 - val_accuracy: 0.6364
Epoch 228/1000
178/178 [==============================] - 0s 179us/step - loss: 2.0242 - accuracy: 0.8090 - val_loss: 0.6934 - 

...

178/178 [==============================] - 0s 176us/step - loss: 1.3559 - accuracy: 0.8989 - val_loss: 0.8322 - val_accuracy: 0.6753
Epoch 773/1000
178/178 [==============================] - 0s 166us/step - loss: 1.6029 - accuracy: 0.8876 - val_loss: 0.8397 - val_accuracy: 0.6234
Epoch 774/1000
178/178 [==============================] - 0s 201us/step - loss: 1.6868 - accuracy: 0.8876 - val_loss: 0.8162 - val_accuracy: 0.6234
Epoch 775/1000
178/178 [==============================] - 0s 180us/step - loss: 1.7825 - accuracy: 0.8708 - val_loss: 0.8743 - val_accuracy: 0.6883
Epoch 776/1000
178/178 [==============================] - 0s 153us/step - loss: 2.2264 - accuracy: 0.8539 - val_loss: 0.8643 - val_accuracy: 0.6494
Epoch 777/1000
178/178 [==============================] - 0s 176us/step - loss: 1.4316 - accuracy: 0.8989 - val_loss: 0.8464 - val_accuracy: 0.6494
Epoch 778/1000
178/178 [==============================] - 0s 201us/step - loss: 1.7816 - accuracy: 0.8708 - val_loss: 0.8405 - 

178/178 [==============================] - 0s 191us/step - loss: 1.7620 - accuracy: 0.8820 - val_loss: 0.8403 - val_accuracy: 0.6494
Epoch 828/1000
178/178 [==============================] - 0s 182us/step - loss: 1.9299 - accuracy: 0.8708 - val_loss: 0.8427 - val_accuracy: 0.6234
Epoch 829/1000
178/178 [==============================] - 0s 202us/step - loss: 2.0149 - accuracy: 0.8708 - val_loss: 0.8534 - val_accuracy: 0.6494
Epoch 830/1000
178/178 [==============================] - 0s 183us/step - loss: 2.1972 - accuracy: 0.8483 - val_loss: 0.8349 - val_accuracy: 0.6494
Epoch 831/1000
178/178 [==============================] - 0s 183us/step - loss: 2.1945 - accuracy: 0.8483 - val_loss: 0.8540 - val_accuracy: 0.6494
Epoch 832/1000
178/178 [==============================] - 0s 202us/step - loss: 1.8606 - accuracy: 0.8764 - val_loss: 0.8390 - val_accuracy: 0.6364
Epoch 833/1000
178/178 [==============================] - 0s 197us/step - loss: 2.0291 - accuracy: 0.8539 - val_loss: 0.8744 - 

178/178 [==============================] - 0s 176us/step - loss: 1.4914 - accuracy: 0.9045 - val_loss: 0.8579 - val_accuracy: 0.6234
Epoch 883/1000
178/178 [==============================] - 0s 184us/step - loss: 1.6770 - accuracy: 0.8820 - val_loss: 0.8875 - val_accuracy: 0.6104
Epoch 884/1000
178/178 [==============================] - 0s 166us/step - loss: 1.9451 - accuracy: 0.8652 - val_loss: 0.8601 - val_accuracy: 0.6623
Epoch 885/1000
178/178 [==============================] - 0s 166us/step - loss: 1.9227 - accuracy: 0.8764 - val_loss: 0.8626 - val_accuracy: 0.6623
Epoch 886/1000
178/178 [==============================] - 0s 177us/step - loss: 2.1874 - accuracy: 0.8596 - val_loss: 0.8607 - val_accuracy: 0.6753
Epoch 887/1000
178/178 [==============================] - 0s 177us/step - loss: 1.7963 - accuracy: 0.8596 - val_loss: 0.8765 - val_accuracy: 0.6364
Epoch 888/1000
178/178 [==============================] - 0s 178us/step - loss: 1.7798 - accuracy: 0.8764 - val_loss: 0.8487 - 

178/178 [==============================] - 0s 186us/step - loss: 1.8589 - accuracy: 0.8596 - val_loss: 0.8657 - val_accuracy: 0.6623
Epoch 938/1000
178/178 [==============================] - 0s 195us/step - loss: 1.4044 - accuracy: 0.9045 - val_loss: 0.8625 - val_accuracy: 0.6364
Epoch 939/1000
178/178 [==============================] - 0s 166us/step - loss: 1.4897 - accuracy: 0.8989 - val_loss: 0.8825 - val_accuracy: 0.6494
Epoch 940/1000
178/178 [==============================] - 0s 167us/step - loss: 1.6574 - accuracy: 0.8933 - val_loss: 0.8588 - val_accuracy: 0.6234
Epoch 941/1000
178/178 [==============================] - 0s 177us/step - loss: 2.2665 - accuracy: 0.8483 - val_loss: 0.8938 - val_accuracy: 0.6364
Epoch 942/1000
178/178 [==============================] - 0s 186us/step - loss: 1.5147 - accuracy: 0.8933 - val_loss: 0.8870 - val_accuracy: 0.6364
Epoch 943/1000
178/178 [==============================] - 0s 167us/step - loss: 1.5001 - accuracy: 0.8933 - val_loss: 0.8872 - 

178/178 [==============================] - 0s 186us/step - loss: 1.7549 - accuracy: 0.8820 - val_loss: 0.8656 - val_accuracy: 0.6623
Epoch 993/1000
178/178 [==============================] - 0s 191us/step - loss: 1.1409 - accuracy: 0.9213 - val_loss: 0.8495 - val_accuracy: 0.6364
Epoch 994/1000
178/178 [==============================] - 0s 213us/step - loss: 1.9321 - accuracy: 0.8652 - val_loss: 0.8650 - val_accuracy: 0.6494
Epoch 995/1000
178/178 [==============================] - 0s 173us/step - loss: 1.1345 - accuracy: 0.9213 - val_loss: 0.8800 - val_accuracy: 0.6364
Epoch 996/1000
178/178 [==============================] - 0s 189us/step - loss: 1.9340 - accuracy: 0.8596 - val_loss: 0.8735 - val_accuracy: 0.6494
Epoch 997/1000
178/178 [==============================] - 0s 180us/step - loss: 2.3502 - accuracy: 0.8427 - val_loss: 0.8714 - val_accuracy: 0.6364
Epoch 998/1000
178/178 [==============================] - 0s 168us/step - loss: 2.1799 - accuracy: 0.8539 - val_loss: 0.8768 - 

In [75]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

77/77 [==============================] - 0s 161us/step
test accuracy: 62.34%


In [63]:
############ Feature selection using lasso ##########

In [31]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [32]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [33]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [34]:
X_train_features = np.vstack((names, X_train))
X_train_features = pd.DataFrame(X_train_features)

In [35]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7002
selected features: 136


In [36]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 354,  403,  633,  634,  647,  672,  861,  874,  883,  991,  994,
         998, 1290, 1372, 1390, 1402, 1521, 1537, 1561, 1576, 1606, 1696,
        1817, 1899, 1947, 2089, 2104, 2106, 2117, 2235, 2283, 2289, 2292,
        2326, 2370, 2420, 2511, 2520, 2546, 2587, 2630, 2666, 2746, 2830,
        2884, 3013, 3046, 3065, 3109, 3161, 3173, 3354, 3391, 3556, 3584,
        3586, 3623, 3686, 3703, 3752, 3817, 3825, 3874, 3882, 4031, 4047,
        4096, 4227, 4278, 4331, 4344, 4350, 4449, 4463, 4544, 4545, 4605,
        4623, 4665, 4736, 4761, 4762, 4842, 4957, 5047, 5068, 5100, 5128,
        5133, 5143, 5189, 5290, 5382, 5487, 5543, 5639, 5658, 5729, 5745,
        5758, 5799, 5824, 5825, 5890, 5955, 5982, 6005, 6024, 6109, 6187,
        6192, 6215, 6285, 6298, 6393, 6421, 6424, 6533, 6581, 6616, 6745,
        6755, 6914, 6926, 6933, 6935, 6941, 6960, 6965, 6972, 6976, 6977,
        6980, 6983, 6984, 6986]])

In [37]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTGGAACAAAAC',
       'TTTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACC',
       'TTTAGACTTAAT', 'TTTAGACTTAATCG',
       'TTTACTGCCGTCATCTTGCCAAAACGTTTCGTAACATGTTCAATTACAAGTCCCATACTTTGCCTCCTAAAAAAATATGTATTTATCTTAATATAACATT',
       'TTTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTC',
       'TTGGAGTAATT',
       'TTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCAACCTTT',
       'TTGCTACTATAA',
       'TTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACCA',
       'TTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGAAATAGTTACGTTAGCATTCAAAGTTATGATTTATTGGCGGTAGGTTCTGATAA',
       'TTGAATAAATTGAATAAAATTACTAAATGTTTGTTTAACACTTTCAATTGCACCATTAACAAAATTTCTAAATGTTTCAGATTTCTTATAAGCAATTGTA',
       'TTATTTTATTACTA', 'TTATCCGAAATTT',
       'TTATATTAAGATAAATACATATTTTTTTAGGAGGCAAAGTATGGGACTTGTAATTGAACATGT

In [38]:
removed_features = X_train_features.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6992, 6993, 6994, 6995, 6996, 6997, 6998, 6999, 7000, 7001],
           dtype='int64', length=6865)

In [39]:
len(removed_features)

6865

In [40]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 136) (255,) (255, 137)


In [41]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [42]:
#### neural network
model_sel1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [43]:
model_sel1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
model_sel1.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 791us/step - loss: 0.6346 - accuracy: 0.6124 - val_loss: 0.6497 - val_accuracy: 0.5714
Epoch 2/1000
178/178 [==============================] - 0s 203us/step - loss: 0.6247 - accuracy: 0.6180 - val_loss: 0.6470 - val_accuracy: 0.5714
Epoch 3/1000
178/178 [==============================] - 0s 220us/step - loss: 0.6157 - accuracy: 0.6180 - val_loss: 0.6454 - val_accuracy: 0.5714
Epoch 4/1000
178/178 [==============================] - 0s 155us/step - loss: 0.6093 - accuracy: 0.6067 - val_loss: 0.6445 - val_accuracy: 0.5584
Epoch 5/1000
178/178 [==============================] - 0s 116us/step - loss: 0.6027 - accuracy: 0.6348 - val_loss: 0.6434 - val_accuracy: 0.5584
Epoch 6/1000
178/178 [==============================] - 0s 120us/step - loss: 0.5970 - accuracy: 0.6517 - val_loss: 0.6419 - val_accuracy: 0.5584
Epoch 7/1000
178/178 [==============================] - 0s 104us/step - loss: 0

178/178 [==============================] - 0s 176us/step - loss: 0.4375 - accuracy: 0.8202 - val_loss: 0.6287 - val_accuracy: 0.5844
Epoch 57/1000
178/178 [==============================] - 0s 172us/step - loss: 0.4354 - accuracy: 0.8146 - val_loss: 0.6292 - val_accuracy: 0.5844
Epoch 58/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4330 - accuracy: 0.8146 - val_loss: 0.6285 - val_accuracy: 0.5844
Epoch 59/1000
178/178 [==============================] - 0s 170us/step - loss: 0.4311 - accuracy: 0.8146 - val_loss: 0.6265 - val_accuracy: 0.5844
Epoch 60/1000
178/178 [==============================] - 0s 134us/step - loss: 0.4286 - accuracy: 0.8146 - val_loss: 0.6265 - val_accuracy: 0.5844
Epoch 61/1000
178/178 [==============================] - 0s 99us/step - loss: 0.4266 - accuracy: 0.8090 - val_loss: 0.6278 - val_accuracy: 0.5844
Epoch 62/1000
178/178 [==============================] - 0s 134us/step - loss: 0.4248 - accuracy: 0.8202 - val_loss: 0.6277 - val_acc

Epoch 112/1000
178/178 [==============================] - 0s 117us/step - loss: 0.3460 - accuracy: 0.8652 - val_loss: 0.6507 - val_accuracy: 0.6494
Epoch 113/1000
178/178 [==============================] - 0s 92us/step - loss: 0.3452 - accuracy: 0.8652 - val_loss: 0.6516 - val_accuracy: 0.6494
Epoch 114/1000
178/178 [==============================] - 0s 144us/step - loss: 0.3452 - accuracy: 0.8708 - val_loss: 0.6520 - val_accuracy: 0.6494
Epoch 115/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3427 - accuracy: 0.8652 - val_loss: 0.6523 - val_accuracy: 0.6494
Epoch 116/1000
178/178 [==============================] - 0s 102us/step - loss: 0.3419 - accuracy: 0.8708 - val_loss: 0.6540 - val_accuracy: 0.6494
Epoch 117/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3404 - accuracy: 0.8708 - val_loss: 0.6551 - val_accuracy: 0.6494
Epoch 118/1000
178/178 [==============================] - 0s 112us/step - loss: 0.3394 - accuracy: 0.8708 - val_los

178/178 [==============================] - 0s 107us/step - loss: 0.2950 - accuracy: 0.8820 - val_loss: 0.6982 - val_accuracy: 0.6623
Epoch 168/1000
178/178 [==============================] - 0s 106us/step - loss: 0.2942 - accuracy: 0.8820 - val_loss: 0.6987 - val_accuracy: 0.6623
Epoch 169/1000
178/178 [==============================] - 0s 297us/step - loss: 0.2932 - accuracy: 0.8820 - val_loss: 0.6997 - val_accuracy: 0.6623
Epoch 170/1000
178/178 [==============================] - 0s 135us/step - loss: 0.2924 - accuracy: 0.8820 - val_loss: 0.7010 - val_accuracy: 0.6623
Epoch 171/1000
178/178 [==============================] - 0s 141us/step - loss: 0.2922 - accuracy: 0.8820 - val_loss: 0.7010 - val_accuracy: 0.6623
Epoch 172/1000
178/178 [==============================] - 0s 128us/step - loss: 0.2918 - accuracy: 0.8820 - val_loss: 0.7025 - val_accuracy: 0.6623
Epoch 173/1000
178/178 [==============================] - 0s 133us/step - loss: 0.2906 - accuracy: 0.8820 - val_loss: 0.7015 - 

178/178 [==============================] - 0s 124us/step - loss: 0.2589 - accuracy: 0.8933 - val_loss: 0.7454 - val_accuracy: 0.6753
Epoch 223/1000
178/178 [==============================] - 0s 116us/step - loss: 0.2578 - accuracy: 0.8933 - val_loss: 0.7464 - val_accuracy: 0.6753
Epoch 224/1000
178/178 [==============================] - 0s 97us/step - loss: 0.2580 - accuracy: 0.8876 - val_loss: 0.7464 - val_accuracy: 0.6753
Epoch 225/1000
178/178 [==============================] - 0s 93us/step - loss: 0.2571 - accuracy: 0.8933 - val_loss: 0.7478 - val_accuracy: 0.6753
Epoch 226/1000
178/178 [==============================] - 0s 102us/step - loss: 0.2566 - accuracy: 0.8933 - val_loss: 0.7476 - val_accuracy: 0.6753
Epoch 227/1000
178/178 [==============================] - 0s 97us/step - loss: 0.2558 - accuracy: 0.8933 - val_loss: 0.7469 - val_accuracy: 0.6753
Epoch 228/1000
178/178 [==============================] - 0s 116us/step - loss: 0.2553 - accuracy: 0.8933 - val_loss: 0.7493 - val

...

Epoch 773/1000
178/178 [==============================] - 0s 96us/step - loss: 0.1225 - accuracy: 0.9719 - val_loss: 1.2160 - val_accuracy: 0.6623
Epoch 774/1000
178/178 [==============================] - 0s 97us/step - loss: 0.1224 - accuracy: 0.9719 - val_loss: 1.2157 - val_accuracy: 0.6623
Epoch 775/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1224 - accuracy: 0.9719 - val_loss: 1.2155 - val_accuracy: 0.6623
Epoch 776/1000
178/178 [==============================] - 0s 121us/step - loss: 0.1221 - accuracy: 0.9719 - val_loss: 1.2163 - val_accuracy: 0.6623
Epoch 777/1000
178/178 [==============================] - 0s 85us/step - loss: 0.1222 - accuracy: 0.9719 - val_loss: 1.2163 - val_accuracy: 0.6623
Epoch 778/1000
178/178 [==============================] - 0s 85us/step - loss: 0.1227 - accuracy: 0.9719 - val_loss: 1.2161 - val_accuracy: 0.6623
Epoch 779/1000
178/178 [==============================] - 0s 82us/step - loss: 0.1219 - accuracy: 0.9719 - val_loss:

178/178 [==============================] - 0s 103us/step - loss: 0.1175 - accuracy: 0.9719 - val_loss: 1.2531 - val_accuracy: 0.6623
Epoch 829/1000
178/178 [==============================] - 0s 98us/step - loss: 0.1179 - accuracy: 0.9719 - val_loss: 1.2548 - val_accuracy: 0.6623
Epoch 830/1000
178/178 [==============================] - 0s 90us/step - loss: 0.1170 - accuracy: 0.9719 - val_loss: 1.2539 - val_accuracy: 0.6623
Epoch 831/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1170 - accuracy: 0.9719 - val_loss: 1.2527 - val_accuracy: 0.6623
Epoch 832/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1174 - accuracy: 0.9719 - val_loss: 1.2547 - val_accuracy: 0.6623
Epoch 833/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1168 - accuracy: 0.9719 - val_loss: 1.2568 - val_accuracy: 0.6623
Epoch 834/1000
178/178 [==============================] - 0s 91us/step - loss: 0.1171 - accuracy: 0.9719 - val_loss: 1.2566 - val

178/178 [==============================] - 0s 127us/step - loss: 0.1136 - accuracy: 0.9719 - val_loss: 1.2890 - val_accuracy: 0.6623
Epoch 884/1000
178/178 [==============================] - 0s 204us/step - loss: 0.1129 - accuracy: 0.9775 - val_loss: 1.2909 - val_accuracy: 0.6623
Epoch 885/1000
178/178 [==============================] - 0s 293us/step - loss: 0.1129 - accuracy: 0.9719 - val_loss: 1.2916 - val_accuracy: 0.6623
Epoch 886/1000
178/178 [==============================] - 0s 153us/step - loss: 0.1128 - accuracy: 0.9719 - val_loss: 1.2936 - val_accuracy: 0.6623
Epoch 887/1000
178/178 [==============================] - 0s 125us/step - loss: 0.1125 - accuracy: 0.9719 - val_loss: 1.2920 - val_accuracy: 0.6623
Epoch 888/1000
178/178 [==============================] - 0s 108us/step - loss: 0.1126 - accuracy: 0.9719 - val_loss: 1.2913 - val_accuracy: 0.6494
Epoch 889/1000
178/178 [==============================] - 0s 95us/step - loss: 0.1127 - accuracy: 0.9775 - val_loss: 1.2939 - v

178/178 [==============================] - 0s 264us/step - loss: 0.1093 - accuracy: 0.9775 - val_loss: 1.3262 - val_accuracy: 0.6623
Epoch 939/1000
178/178 [==============================] - 0s 178us/step - loss: 0.1088 - accuracy: 0.9775 - val_loss: 1.3256 - val_accuracy: 0.6623
Epoch 940/1000
178/178 [==============================] - 0s 145us/step - loss: 0.1093 - accuracy: 0.9775 - val_loss: 1.3282 - val_accuracy: 0.6623
Epoch 941/1000
178/178 [==============================] - 0s 95us/step - loss: 0.1083 - accuracy: 0.9775 - val_loss: 1.3274 - val_accuracy: 0.6623
Epoch 942/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1090 - accuracy: 0.9775 - val_loss: 1.3281 - val_accuracy: 0.6623
Epoch 943/1000
178/178 [==============================] - 0s 121us/step - loss: 0.1085 - accuracy: 0.9775 - val_loss: 1.3278 - val_accuracy: 0.6623
Epoch 944/1000
178/178 [==============================] - 0s 97us/step - loss: 0.1086 - accuracy: 0.9775 - val_loss: 1.3281 - va

178/178 [==============================] - 0s 85us/step - loss: 0.1056 - accuracy: 0.9775 - val_loss: 1.3661 - val_accuracy: 0.6623
Epoch 994/1000
178/178 [==============================] - 0s 86us/step - loss: 0.1054 - accuracy: 0.9775 - val_loss: 1.3679 - val_accuracy: 0.6623
Epoch 995/1000
178/178 [==============================] - 0s 81us/step - loss: 0.1056 - accuracy: 0.9775 - val_loss: 1.3659 - val_accuracy: 0.6623
Epoch 996/1000
178/178 [==============================] - 0s 90us/step - loss: 0.1060 - accuracy: 0.9775 - val_loss: 1.3666 - val_accuracy: 0.6623
Epoch 997/1000
178/178 [==============================] - 0s 79us/step - loss: 0.1053 - accuracy: 0.9775 - val_loss: 1.3642 - val_accuracy: 0.6623
Epoch 998/1000
178/178 [==============================] - 0s 81us/step - loss: 0.1046 - accuracy: 0.9775 - val_loss: 1.3664 - val_accuracy: 0.6623
Epoch 999/1000
178/178 [==============================] - 0s 82us/step - loss: 0.1047 - accuracy: 0.9775 - val_loss: 1.3662 - val_acc

In [45]:
acc_test_sel1 = model_sel1.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel1*100))

77/77 [==============================] - 0s 112us/step
test accuracy: 66.23%


In [46]:
#### improve neural network
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [47]:
model_sel2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 1ms/step - loss: 0.7095 - accuracy: 0.3652 - val_loss: 0.6832 - val_accuracy: 0.4026
Epoch 2/1000
178/178 [==============================] - 0s 74us/step - loss: 0.6912 - accuracy: 0.4326 - val_loss: 0.6759 - val_accuracy: 0.4935
Epoch 3/1000
178/178 [==============================] - 0s 96us/step - loss: 0.6746 - accuracy: 0.4831 - val_loss: 0.6721 - val_accuracy: 0.4935
Epoch 4/1000
178/178 [==============================] - 0s 106us/step - loss: 0.6639 - accuracy: 0.5337 - val_loss: 0.6707 - val_accuracy: 0.5065
Epoch 5/1000
178/178 [==============================] - 0s 99us/step - loss: 0.6536 - accuracy: 0.5506 - val_loss: 0.6695 - val_accuracy: 0.4935
Epoch 6/1000
178/178 [==============================] - 0s 107us/step - loss: 0.6458 - accuracy: 0.5618 - val_loss: 0.6687 - val_accuracy: 0.5065
Epoch 7/1000
178/178 [==============================] - 0s 128us/step - loss: 0.6404

178/178 [==============================] - 0s 105us/step - loss: 0.4553 - accuracy: 0.8090 - val_loss: 0.6489 - val_accuracy: 0.5714
Epoch 57/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4506 - accuracy: 0.8090 - val_loss: 0.6508 - val_accuracy: 0.5714
Epoch 58/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4476 - accuracy: 0.8090 - val_loss: 0.6501 - val_accuracy: 0.5714
Epoch 59/1000
178/178 [==============================] - 0s 99us/step - loss: 0.4444 - accuracy: 0.8090 - val_loss: 0.6506 - val_accuracy: 0.5714
Epoch 60/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4417 - accuracy: 0.8090 - val_loss: 0.6514 - val_accuracy: 0.5714
Epoch 61/1000
178/178 [==============================] - 0s 97us/step - loss: 0.4382 - accuracy: 0.8090 - val_loss: 0.6514 - val_accuracy: 0.5714
Epoch 62/1000
178/178 [==============================] - 0s 88us/step - loss: 0.4349 - accuracy: 0.8090 - val_loss: 0.6499 - val_accura

178/178 [==============================] - 0s 100us/step - loss: 0.3224 - accuracy: 0.8708 - val_loss: 0.7376 - val_accuracy: 0.6494
Epoch 113/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3195 - accuracy: 0.8764 - val_loss: 0.7401 - val_accuracy: 0.6494
Epoch 114/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3184 - accuracy: 0.8764 - val_loss: 0.7460 - val_accuracy: 0.6364
Epoch 115/1000
178/178 [==============================] - 0s 82us/step - loss: 0.3165 - accuracy: 0.8764 - val_loss: 0.7471 - val_accuracy: 0.6364
Epoch 116/1000
178/178 [==============================] - 0s 83us/step - loss: 0.3151 - accuracy: 0.8708 - val_loss: 0.7456 - val_accuracy: 0.6494
Epoch 117/1000
178/178 [==============================] - 0s 80us/step - loss: 0.3128 - accuracy: 0.8652 - val_loss: 0.7496 - val_accuracy: 0.6494
Epoch 118/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3110 - accuracy: 0.8708 - val_loss: 0.7533 - val_ac

Epoch 168/1000
178/178 [==============================] - 0s 94us/step - loss: 0.2461 - accuracy: 0.9101 - val_loss: 0.8758 - val_accuracy: 0.6494
Epoch 169/1000
178/178 [==============================] - 0s 88us/step - loss: 0.2462 - accuracy: 0.9101 - val_loss: 0.8780 - val_accuracy: 0.6494
Epoch 170/1000
178/178 [==============================] - 0s 82us/step - loss: 0.2451 - accuracy: 0.9101 - val_loss: 0.8788 - val_accuracy: 0.6494
Epoch 171/1000
178/178 [==============================] - 0s 91us/step - loss: 0.2436 - accuracy: 0.9045 - val_loss: 0.8867 - val_accuracy: 0.6494
Epoch 172/1000
178/178 [==============================] - 0s 85us/step - loss: 0.2425 - accuracy: 0.9157 - val_loss: 0.8882 - val_accuracy: 0.6494
Epoch 173/1000
178/178 [==============================] - 0s 87us/step - loss: 0.2417 - accuracy: 0.9045 - val_loss: 0.8891 - val_accuracy: 0.6494
Epoch 174/1000
178/178 [==============================] - 0s 85us/step - loss: 0.2422 - accuracy: 0.9157 - val_loss: 0

178/178 [==============================] - 0s 101us/step - loss: 0.1952 - accuracy: 0.9438 - val_loss: 1.0181 - val_accuracy: 0.6494
Epoch 224/1000
178/178 [==============================] - 0s 112us/step - loss: 0.1950 - accuracy: 0.9382 - val_loss: 1.0200 - val_accuracy: 0.6494
Epoch 225/1000
178/178 [==============================] - 0s 83us/step - loss: 0.1935 - accuracy: 0.9438 - val_loss: 1.0239 - val_accuracy: 0.6623
Epoch 226/1000
178/178 [==============================] - 0s 84us/step - loss: 0.1935 - accuracy: 0.9270 - val_loss: 1.0242 - val_accuracy: 0.6623
Epoch 227/1000
178/178 [==============================] - 0s 83us/step - loss: 0.1925 - accuracy: 0.9438 - val_loss: 1.0283 - val_accuracy: 0.6623
Epoch 228/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1911 - accuracy: 0.9326 - val_loss: 1.0287 - val_accuracy: 0.6494
Epoch 229/1000
178/178 [==============================] - 0s 88us/step - loss: 0.1904 - accuracy: 0.9438 - val_loss: 1.0352 - val_

...

Epoch 775/1000
178/178 [==============================] - 0s 96us/step - loss: 0.0700 - accuracy: 0.9831 - val_loss: 2.0766 - val_accuracy: 0.6623
Epoch 776/1000
178/178 [==============================] - 0s 102us/step - loss: 0.0679 - accuracy: 0.9775 - val_loss: 2.0736 - val_accuracy: 0.6623
Epoch 777/1000
178/178 [==============================] - 0s 90us/step - loss: 0.0680 - accuracy: 0.9831 - val_loss: 2.0745 - val_accuracy: 0.6623
Epoch 778/1000
178/178 [==============================] - 0s 108us/step - loss: 0.0691 - accuracy: 0.9775 - val_loss: 2.1083 - val_accuracy: 0.6364
Epoch 779/1000
178/178 [==============================] - 0s 107us/step - loss: 0.0711 - accuracy: 0.9831 - val_loss: 2.0821 - val_accuracy: 0.6623
Epoch 780/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0685 - accuracy: 0.9831 - val_loss: 2.0729 - val_accuracy: 0.6623
Epoch 781/1000
178/178 [==============================] - 0s 93us/step - loss: 0.0697 - accuracy: 0.9775 - val_loss

178/178 [==============================] - 0s 95us/step - loss: 0.0652 - accuracy: 0.9831 - val_loss: 2.1289 - val_accuracy: 0.6623
Epoch 831/1000
178/178 [==============================] - 0s 92us/step - loss: 0.0653 - accuracy: 0.9831 - val_loss: 2.1425 - val_accuracy: 0.6623
Epoch 832/1000
178/178 [==============================] - 0s 90us/step - loss: 0.0670 - accuracy: 0.9831 - val_loss: 2.1202 - val_accuracy: 0.6623
Epoch 833/1000
178/178 [==============================] - 0s 92us/step - loss: 0.0673 - accuracy: 0.9831 - val_loss: 2.1283 - val_accuracy: 0.6623
Epoch 834/1000
178/178 [==============================] - 0s 97us/step - loss: 0.0671 - accuracy: 0.9831 - val_loss: 2.1361 - val_accuracy: 0.6623
Epoch 835/1000
178/178 [==============================] - 0s 92us/step - loss: 0.0642 - accuracy: 0.9831 - val_loss: 2.1531 - val_accuracy: 0.6623
Epoch 836/1000
178/178 [==============================] - 0s 89us/step - loss: 0.0667 - accuracy: 0.9831 - val_loss: 2.1393 - val_acc

Epoch 886/1000
178/178 [==============================] - 0s 91us/step - loss: 0.0642 - accuracy: 0.9831 - val_loss: 2.2093 - val_accuracy: 0.6623
Epoch 887/1000
178/178 [==============================] - 0s 92us/step - loss: 0.0628 - accuracy: 0.9831 - val_loss: 2.2031 - val_accuracy: 0.6623
Epoch 888/1000
178/178 [==============================] - 0s 97us/step - loss: 0.0636 - accuracy: 0.9831 - val_loss: 2.1677 - val_accuracy: 0.6623
Epoch 889/1000
178/178 [==============================] - 0s 91us/step - loss: 0.0642 - accuracy: 0.9775 - val_loss: 2.2065 - val_accuracy: 0.6623
Epoch 890/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0651 - accuracy: 0.9831 - val_loss: 2.2220 - val_accuracy: 0.6623
Epoch 891/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0644 - accuracy: 0.9831 - val_loss: 2.2228 - val_accuracy: 0.6623
Epoch 892/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0667 - accuracy: 0.9831 - val_loss: 2

178/178 [==============================] - 0s 101us/step - loss: 0.0626 - accuracy: 0.9831 - val_loss: 2.2666 - val_accuracy: 0.6623
Epoch 942/1000
178/178 [==============================] - 0s 129us/step - loss: 0.0618 - accuracy: 0.9831 - val_loss: 2.2760 - val_accuracy: 0.6494
Epoch 943/1000
178/178 [==============================] - 0s 106us/step - loss: 0.0619 - accuracy: 0.9831 - val_loss: 2.2623 - val_accuracy: 0.6623
Epoch 944/1000
178/178 [==============================] - 0s 100us/step - loss: 0.0612 - accuracy: 0.9831 - val_loss: 2.2494 - val_accuracy: 0.6623
Epoch 945/1000
178/178 [==============================] - 0s 103us/step - loss: 0.0613 - accuracy: 0.9831 - val_loss: 2.2732 - val_accuracy: 0.6623
Epoch 946/1000
178/178 [==============================] - 0s 114us/step - loss: 0.0638 - accuracy: 0.9831 - val_loss: 2.2663 - val_accuracy: 0.6623
Epoch 947/1000
178/178 [==============================] - 0s 106us/step - loss: 0.0639 - accuracy: 0.9831 - val_loss: 2.2284 - 

178/178 [==============================] - 0s 101us/step - loss: 0.0616 - accuracy: 0.9831 - val_loss: 2.2907 - val_accuracy: 0.6623
Epoch 997/1000
178/178 [==============================] - 0s 115us/step - loss: 0.0604 - accuracy: 0.9831 - val_loss: 2.3093 - val_accuracy: 0.6623
Epoch 998/1000
178/178 [==============================] - 0s 101us/step - loss: 0.0611 - accuracy: 0.9831 - val_loss: 2.3179 - val_accuracy: 0.6623
Epoch 999/1000
178/178 [==============================] - 0s 96us/step - loss: 0.0599 - accuracy: 0.9831 - val_loss: 2.3375 - val_accuracy: 0.6623
Epoch 1000/1000
178/178 [==============================] - 0s 90us/step - loss: 0.0636 - accuracy: 0.9831 - val_loss: 2.3239 - val_accuracy: 0.6623


In [49]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

77/77 [==============================] - 0s 110us/step
test accuracy: 66.23%


In [50]:
############ Logistic Regression ############

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [52]:
###### logistics
log = LogisticRegression(random_state=123)
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
y_pred = log.predict(X_test)
print('combination logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

combination logistic test accuracy 62.34%


In [54]:
############## Random Forest ##############

In [55]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [56]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

combination test accuracy: 62.34%


In [57]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [58]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.80555556 0.63888889 0.77777778 0.73529412]
0.7526143790849673
